In [2]:
minSup = 0.7
minConf = 0.5

In [3]:
Dataset = 'Dataset1.dat'
Transactions = []
ItemswSpprt = defaultdict(int)

In [4]:
def createTransaction(Dataset):
    with open(Dataset) as f:
        for line in f:
            items = set()
            for item in map(int, line.split()):
                items.add(item) 
            Transactions.append(copy.deepcopy(items))
    #return Transactions

In [5]:
def calculateSup(curCandidates, minSup):
        yerel = defaultdict(int)

        for items in Transactions:
                for item in curCandidates:
                    if item < items :
                        ItemswSpprt[frozenset(item)] += 1
                        yerel[frozenset(item)] += 1

        return yerel 
    
def subsets(arr):
    return chain(*[combinations(arr, i + 1) for i, a in enumerate(arr)])

In [6]:
def getItems (yerel) :
    results = set()
    for item, count in yerel.items():
                support = float(count)/len(Transactions)

                if support >= minSup:
                        results.add(item)
    return results

In [7]:
def genCandidates(items, length):
        return set([i.union(j) for i in items for j in items if len(i.union(j)) == length])
    
def genItems(items):
    FrequentItems = []
    for key, value in items.items():
        FrequentItems.extend([(tuple(item), float(ItemswSpprt[item])/len(Transactions)) for item in value])
    return FrequentItems

In [8]:
def apriori( minSup, Dataset):
    
    createTransaction(Dataset)
    BigSet = dict()
    BiggerSet = set()
    curCandidates = []
    
    for items in Transactions :
        BiggerSet = BiggerSet.union(items)
    
    for item in BiggerSet :
        curCandidates.append(set([item]))
    
    curYerel = calculateSup(curCandidates,minSup)
    curResults = getItems(curYerel)
    
    k = 2
    while(curResults != set([])):
        BigSet[k-1] = curResults
        curResults = getItems(calculateSup(genCandidates(curResults, k),minSup))
        k = k + 1
    return BigSet

In [9]:
def Closeditems(BigSet):
    closeds = copy.deepcopy(BigSet[len(BigSet)])
    for i in reversed(range(1,len(BigSet))):
        for subset in BigSet[i]:
            flag = True
            for superset in BigSet[i+1]:
                if subset < superset :
                    flag = False
            if flag == True :
                closeds.add(subset)
    return closeds

In [10]:
def Maximalitem(BigSet):
    maximals = copy.deepcopy(BigSet[len(BigSet)])
    for i in reversed(range(1,len(BigSet))):
        for subset in BigSet[i]:
            flag = True
            for superset in BigSet[i+1]:
                if subset < superset and ItemswSpprt[subset] == ItemswSpprt[superset] :
                    flag = False
            if flag == True :
                maximals.add(subset)
    return maximals

In [11]:
def AssocRules(BigSet,minConf):
    AssociationRules = []
    for key, value in list(BigSet.items())[1:]:
        for item in value:
            Subsets = map(frozenset, [x for x in subsets(item)])
            for element in Subsets:
                remain = item.difference(element)
                if len(remain) > 0:
                    confidence = float(ItemswSpprt[item])/float(ItemswSpprt[element])
                    lift = float(ItemswSpprt[item]*float(len(Transactions)))/(float(ItemswSpprt[element])*float(ItemswSpprt[remain]))
                    allConf = float(ItemswSpprt[item])/max((float(ItemswSpprt[element]),float(ItemswSpprt[remain])))
                    cosine = float(ItemswSpprt[item])/math.sqrt((float(ItemswSpprt[element])*float(ItemswSpprt[remain])))
                    if confidence >= minConf:
                        AssociationRules.append(((tuple(element), tuple(remain)),confidence,lift,allConf,cosine))
    return AssociationRules

In [12]:
BigSet = apriori(minSup,Dataset)
frequentItems = genItems(BigSet)

print ("\n FREQUENT ITEMSET:")
#printFreqItemset(frequentItems)
for item, support in frequentItems:
    print (" %s---------------Support %.3f" % (str(item), support))

gecici1 = dict()
gecici1[0] = Maximalitem(BigSet)
maximals = genItems(gecici1)
print ("\n MAXIMAL FREQ. ITEMSET:")
for item, support in maximals:
    print (" %s---------------Support %.3f" % (str(item), support))

gecici2 = dict()
gecici2[0] = Closeditems(BigSet)
closeds = genItems(gecici2)
print ("\n CLOSED FREQ. ITEMSET:")
for item, support in closeds:
    print (" %s---------------Support %.3f" % (str(item), support))


AssociationRules = AssocRules(BigSet,minConf)
print ("\n ASSOCIATION RULES:")

for rule, confidence, lift, allConf,cosine in AssociationRules:
    ilk, son = rule
    print ("Rule: %s--->%s , Conf:--- %.3f , Lift:-- %.3f , All_conf:-- %.3f , Cosine:-- %.3f" % (str(ilk), str(son), confidence, lift, allConf,cosine))


 FREQUENT ITEMSET:
 (2,)---------------Support 0.894
 (11,)---------------Support 0.871
 (17,)---------------Support 0.895
 (14,)---------------Support 0.817
 (9,)---------------Support 0.876
 (19,)---------------Support 0.828
 (8,)---------------Support 0.875
 (7,)---------------Support 0.887
 (3,)---------------Support 0.861
 (10,)---------------Support 0.866
 (1,)---------------Support 0.867
 (6,)---------------Support 0.822
 (18,)---------------Support 0.891
 (13,)---------------Support 0.896
 (5,)---------------Support 0.825
 (15,)---------------Support 0.889
 (12,)---------------Support 0.800
 (16,)---------------Support 0.871
 (4,)---------------Support 0.858
 (0,)---------------Support 0.895
 (8, 2)---------------Support 0.786
 (2, 3)---------------Support 0.771
 (10, 13)---------------Support 0.782
 (8, 7)---------------Support 0.780
 (14, 7)---------------Support 0.730
 (5, 13)---------------Support 0.739
 (2, 11)---------------Support 0.780
 (1, 10)---------------Support 0.